# Download lab

In [3]:
!rm -rf mapreduce.zip
!rm -rf mapreduce

In [4]:
!wget https://www.dropbox.com/s/74ddi95lgixagi4/mapreduce.zip?dl=0 -O mapreduce.zip

--2020-08-25 17:37:32--  https://www.dropbox.com/s/74ddi95lgixagi4/mapreduce.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.1, 2620:100:6032:1::a27d:5201
Connecting to www.dropbox.com (www.dropbox.com)|162.125.82.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/74ddi95lgixagi4/mapreduce.zip [following]
--2020-08-25 17:37:33--  https://www.dropbox.com/s/raw/74ddi95lgixagi4/mapreduce.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uce53a2e5eee18e111d675b9a5cc.dl.dropboxusercontent.com/cd/0/inline/A-Hr_jZDm9sTg2ay5tKI8pLJdqPtWzcxQquIZ5Sxm7iKrLaQkeGJpbSXJODUiwnpT_-ZXfYF0QsKkcp_9uNTd0AKm8iPI-w2fxkijb5Yayz8ojIZjX43SRUzPPVWpsDpg44/file# [following]
--2020-08-25 17:37:33--  https://uce53a2e5eee18e111d675b9a5cc.dl.dropboxusercontent.com/cd/0/inline/A-Hr_jZDm9sTg2ay5tKI8pLJdqPtWzcxQquIZ5Sxm7iKrLaQkeGJpbSXJODUiwnpT_-ZXfYF0QsKkcp_9uNTd0AKm8iPI-w2fxkijb5

In [5]:
!unzip mapreduce.zip
!rm -rf mapreduce.zip
!rm -rf __MACOSX/

Archive:  mapreduce.zip
   creating: mapreduce/
  inflating: mapreduce/.DS_Store     
   creating: __MACOSX/
   creating: __MACOSX/mapreduce/
  inflating: __MACOSX/mapreduce/._.DS_Store  
   creating: mapreduce/ex0/
  inflating: mapreduce/ex0/mapper.py  
   creating: __MACOSX/mapreduce/ex0/
  inflating: __MACOSX/mapreduce/ex0/._mapper.py  
  inflating: mapreduce/ex0/reducer.py  
  inflating: __MACOSX/mapreduce/ex0/._reducer.py  
  inflating: __MACOSX/mapreduce/._ex0  
   creating: mapreduce/ex1/
  inflating: mapreduce/ex1/mapper.py  
   creating: __MACOSX/mapreduce/ex1/
  inflating: __MACOSX/mapreduce/ex1/._mapper.py  
  inflating: mapreduce/ex1/reducer.py  
  inflating: __MACOSX/mapreduce/ex1/._reducer.py  
  inflating: __MACOSX/mapreduce/._ex1  
   creating: mapreduce/ex2/
  inflating: mapreduce/ex2/mapper.py  
   creating: __MACOSX/mapreduce/ex2/
  inflating: __MACOSX/mapreduce/ex2/._mapper.py  
  inflating: mapreduce/ex2/reducer.py  
  inflating: __MACOSX/mapreduce/ex2/._reducer.py

In [6]:
!ls -al mapreduce

total 12
drwxr-xr-x. 8 admin admin  104 Sep  2  2018 .
drwxrwxr-x. 4 admin admin   65 Aug 25 17:39 ..
-rw-r--r--. 1 admin admin 8196 Sep  1  2018 .DS_Store
drwxr-xr-x. 2 admin admin   41 Sep  2  2018 ex0
drwxr-xr-x. 2 admin admin   41 Sep  2  2018 ex1
drwxr-xr-x. 2 admin admin   41 Sep  2  2018 ex2
drwxr-xr-x. 2 admin admin   41 Sep  2  2018 ex3
drwxr-xr-x. 2 admin admin   57 Sep  2  2018 ex4_student
drwxr-xr-x. 2 admin admin   41 Sep  2  2018 word_count


In [7]:
!chmod +x mapreduce/ex0/mapper.py
!chmod +x mapreduce/ex0/reducer.py
!chmod +x mapreduce/ex1/mapper.py
!chmod +x mapreduce/ex1/reducer.py
!chmod +x mapreduce/ex2/mapper.py
!chmod +x mapreduce/ex2/reducer.py
!chmod +x mapreduce/ex3/mapper.py
!chmod +x mapreduce/ex3/reducer.py
!chmod +x mapreduce/ex4_student/mapper_student.py
!chmod +x mapreduce/ex4_student/reducer_student.py
!chmod +x mapreduce/word_count/mapper.py
!chmod +x mapreduce/word_count/reducer.py

# Clear to initial stage

In [8]:
!hdfs dfs -rm -r -f output/*

# Word count (using python on localnode) [word_count]

In [9]:
cat mapreduce/word_count/mapper.py

#!/usr/bin/env python

import sys

# input comes from STDIN (standard input)
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()
    # split the line into words
    words = line.split()
    # increase counters
    for word in words:
        # write the results to STDOUT (standard output);
        # what we output here will be the input for the
        # Reduce step, i.e. the input for reducer.py
        #
        # tab-delimited; the trivial word count is 1
        print('%s\t%s' % (word.lower(), 1))


In [10]:
cat mapreduce/word_count/reducer.py

#!/usr/bin/env python

from operator import itemgetter
import sys

current_word = None
current_count = 0
word = None

# input comes from STDIN
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()

    # parse the input we got from mapper.py
    word, count = line.split('\t', 1)

    # convert count (currently a string) to int
    try:
        count = int(count)
    except ValueError:
        # count was not a number, so silently
        # ignore/discard this line
        continue

    # this IF-switch only works because Hadoop sorts map output
    # by key (here: word) before it is passed to the reducer
    if current_word == word:
        current_count += count
    else:
        if current_word:
            # write result to STDOUT
            print('%s\t%s' % (current_word, current_count))
        current_count = count
        current_word = word

# do not forget to output the last word if needed!
if current_word == word:
    print('%s\t%s' % (c

In [11]:
cat ~/data/fox_lyrics.txt 

Dog goes woof
Cat goes meow
Bird goes tweet
And mouse goes squeek
Cow goes moo
Frog goes croak
And the elephant goes toot
Ducks say quack
And fish go blub
And the seal goes ow ow ow


In [12]:
cat ~/data/fox_lyrics.txt | mapreduce/word_count/mapper.py

dog	1
goes	1
woof	1
cat	1
goes	1
meow	1
bird	1
goes	1
tweet	1
and	1
mouse	1
goes	1
squeek	1
cow	1
goes	1
moo	1
frog	1
goes	1
croak	1
and	1
the	1
elephant	1
goes	1
toot	1
ducks	1
say	1
quack	1
and	1
fish	1
go	1
blub	1
and	1
the	1
seal	1
goes	1
ow	1
ow	1
ow	1


In [13]:
cat ~/data/fox_lyrics.txt | mapreduce/word_count/mapper.py | sort

and	1
and	1
and	1
and	1
bird	1
blub	1
cat	1
cow	1
croak	1
dog	1
ducks	1
elephant	1
fish	1
frog	1
go	1
goes	1
goes	1
goes	1
goes	1
goes	1
goes	1
goes	1
goes	1
meow	1
moo	1
mouse	1
ow	1
ow	1
ow	1
quack	1
say	1
seal	1
squeek	1
the	1
the	1
toot	1
tweet	1
woof	1


In [14]:
cat ~/data/fox_lyrics.txt | mapreduce/word_count/mapper.py | sort | mapreduce/word_count/reducer.py

and	4
bird	1
blub	1
cat	1
cow	1
croak	1
dog	1
ducks	1
elephant	1
fish	1
frog	1
go	1
goes	8
meow	1
moo	1
mouse	1
ow	3
quack	1
say	1
seal	1
squeek	1
the	2
toot	1
tweet	1
woof	1


# Word count (using python on mapreduce)

In [15]:
!hdfs dfs -rm -r output/mapreduce/word_count

rm: `output/mapreduce/word_count': No such file or directory


In [16]:
!hadoop jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
-files mapreduce/word_count/mapper.py,mapreduce/word_count/reducer.py \
-mapper "python mapper.py" \
-reducer "python reducer.py" \
-input "data/fox_lyrics.txt" \
-output "output/mapreduce/word_count" \
-numReduceTasks 2 

packageJobJar: [] [/opt/cloudera/parcels/CDH-6.3.2-1.cdh6.3.2.p0.1605554/jars/hadoop-streaming-3.0.0-cdh6.3.2.jar] /tmp/streamjob4407548217863901733.jar tmpDir=null
20/08/25 17:39:38 INFO client.RMProxy: Connecting to ResourceManager at cdh-all.asia-east1-b.c.kao-project-286007.internal/10.140.0.4:8032
20/08/25 17:39:38 INFO client.RMProxy: Connecting to ResourceManager at cdh-all.asia-east1-b.c.kao-project-286007.internal/10.140.0.4:8032
20/08/25 17:39:38 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /user/admin/.staging/job_1598350941451_0028
20/08/25 17:39:39 INFO mapred.FileInputFormat: Total input files to process : 1
20/08/25 17:39:39 INFO mapreduce.JobSubmitter: number of splits:2
20/08/25 17:39:39 INFO Configuration.deprecation: yarn.resourcemanager.system-metrics-publisher.enabled is deprecated. Instead, use yarn.system-metrics-publisher.enabled
20/08/25 17:39:39 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1598350941451_0028
20/08/25 17

## Output

In [20]:
!hdfs dfs -ls output/mapreduce/word_count/

Found 3 items
-rw-r--r--   3 admin admin          0 2020-08-25 17:39 output/mapreduce/word_count/_SUCCESS
-rw-r--r--   3 admin admin         93 2020-08-25 17:39 output/mapreduce/word_count/part-00000
-rw-r--r--   3 admin admin         82 2020-08-25 17:39 output/mapreduce/word_count/part-00001


In [21]:
!hdfs dfs -cat output/mapreduce/word_count/part-00000

and	4
bird	1
blub	1
cow	1
elephant	1
moo	1
mouse	1
quack	1
say	1
seal	1
the	2
tweet	1
woof	1


In [22]:
!hdfs dfs -cat output/mapreduce/word_count/part-00001

cat	1
croak	1
dog	1
ducks	1
fish	1
frog	1
go	1
goes	8
meow	1
ow	3
squeek	1
toot	1


## Show job run in Jobbrowser

In [25]:
%%html
<a href="cdh-all:8088">YARN HISTORY SERVER LINK</a>

## Show job run in Hue

In [26]:
%%html
<a href="cdh-all:8889">HUE LINK</a>

# Find total calling out duration per user (ex0)

In [27]:
cat mapreduce/ex0/mapper.py

#!/usr/bin/env python

import sys
for line in sys.stdin:
	# remove leading and trailing whitespace
	line = line.strip()
	rows = line.split(",")
	(name1,phone1,name2,phone2,timestamp_ms,timestamp_str,duration_m) = tuple(rows)
	#print(rows)
	# write the results to STDOUT (standard output);
	# what we output here will be the input for the
	# Reduce step, i.e. the input for reducer.py
	try: 
		print('{}\t{}'.format(name1, int(duration_m)))
	except ValueError as e:
		continue	



In [28]:
cat mapreduce/ex0/reducer.py

#!/usr/bin/env python

from operator import itemgetter
import sys

current_key = None
sum = 0

# input comes from STDIN
for line in sys.stdin:
	# remove leading and trailing whitespace
	line = line.strip()
	row = line.split("\t")
	(key,value) = row

	try: 
		value = int(value)
	except ValueError:
		continue

	# this IF-switch only works because Hadoop sorts map output
	# by key (here: word) before it is passed to the reducer
	if current_key == key:
		sum += value
	else:
		if current_key:
	 		# write result to STDOUT
			print('{}\t{}'.format(current_key,sum))
		current_key = key
		sum = value

#Last key
if current_key == key:
	print('{}\t{}'.format(current_key,sum))



In [29]:
!hdfs dfs -rm -r output/mapreduce/ex0

rm: `output/mapreduce/ex0': No such file or directory


In [30]:
!hadoop jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
-files mapreduce/ex0/mapper.py,mapreduce/ex0/reducer.py \
-mapper "python mapper.py" \
-reducer "python reducer.py" \
-input "data/cdr.csv" \
-output "output/mapreduce/ex0" \
-numReduceTasks 2 

packageJobJar: [] [/opt/cloudera/parcels/CDH-6.3.2-1.cdh6.3.2.p0.1605554/jars/hadoop-streaming-3.0.0-cdh6.3.2.jar] /tmp/streamjob4004633308679871776.jar tmpDir=null
20/08/25 17:45:34 INFO client.RMProxy: Connecting to ResourceManager at cdh-all.asia-east1-b.c.kao-project-286007.internal/10.140.0.4:8032
20/08/25 17:45:34 INFO client.RMProxy: Connecting to ResourceManager at cdh-all.asia-east1-b.c.kao-project-286007.internal/10.140.0.4:8032
20/08/25 17:45:34 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /user/admin/.staging/job_1598350941451_0055
20/08/25 17:45:34 INFO mapred.FileInputFormat: Total input files to process : 1
20/08/25 17:45:34 INFO mapreduce.JobSubmitter: number of splits:2
20/08/25 17:45:34 INFO Configuration.deprecation: yarn.resourcemanager.system-metrics-publisher.enabled is deprecated. Instead, use yarn.system-metrics-publisher.enabled
20/08/25 17:45:35 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1598350941451_0055
20/08/25 17

## Output

In [31]:
!hdfs dfs -ls output/mapreduce/ex0/

Found 3 items
-rw-r--r--   3 admin admin          0 2020-08-25 17:46 output/mapreduce/ex0/_SUCCESS
-rw-r--r--   3 admin admin         51 2020-08-25 17:46 output/mapreduce/ex0/part-00000
-rw-r--r--   3 admin admin         90 2020-08-25 17:46 output/mapreduce/ex0/part-00001


In [32]:
!hdfs dfs -cat output/mapreduce/ex0/part-00000

boss	45407
it	142842
kao	42105
non	50801
por	69700


In [33]:
!hdfs dfs -cat output/mapreduce/ex0/part-00001

abe	178699
bear	86826
boat	58248
can	39184
dear	74793
fong	175763
punpun	53608
vate	70130


# Find total calling duration of each couple (ex1)

In [34]:
cat mapreduce/ex1/mapper.py

#!/usr/bin/env python

import sys
for line in sys.stdin:
	# remove leading and trailing whitespace
	line = line.strip()
	rows = line.split(",")
	(name1,phone1,name2,phone2,timestamp_ms,timestamp_str,duration_m) = tuple(rows)
	#print(rows)
	# write the results to STDOUT (standard output);
	# what we output here will be the input for the
	# Reduce step, i.e. the input for reducer.py
	try: 
		print('({},{})\t{}'.format(name1,name2, int(duration_m)))
	except ValueError as e:
		continue	



In [35]:
cat mapreduce/ex1/reducer.py

#!/usr/bin/env python

from operator import itemgetter
import sys

current_key = None
sum = 0

# input comes from STDIN
for line in sys.stdin:
	# remove leading and trailing whitespace
	line = line.strip()
	row = line.split("\t")
	(key,value) = row

	try: 
		value = int(value)
	except ValueError:
		continue

	# this IF-switch only works because Hadoop sorts map output
	# by key (here: word) before it is passed to the reducer
	if current_key == (key):
		sum += value
	else:
		if current_key:
	 		# write result to STDOUT
			print('{}\t{}'.format(current_key,sum))
		current_key = key
		sum = value


if current_key == key:
	print('{}\t{}'.format(current_key,sum))



In [36]:
!hdfs dfs -rm -r output/mapreduce/ex1

rm: `output/mapreduce/ex1': No such file or directory


In [37]:
!hadoop jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
-files mapreduce/ex1/mapper.py,mapreduce/ex1/reducer.py \
-mapper "python mapper.py" \
-reducer "python reducer.py" \
-input "data/cdr.csv" \
-output "output/mapreduce/ex1" \
-numReduceTasks 2 

packageJobJar: [] [/opt/cloudera/parcels/CDH-6.3.2-1.cdh6.3.2.p0.1605554/jars/hadoop-streaming-3.0.0-cdh6.3.2.jar] /tmp/streamjob3634889364828275797.jar tmpDir=null
20/08/25 17:46:42 INFO client.RMProxy: Connecting to ResourceManager at cdh-all.asia-east1-b.c.kao-project-286007.internal/10.140.0.4:8032
20/08/25 17:46:42 INFO client.RMProxy: Connecting to ResourceManager at cdh-all.asia-east1-b.c.kao-project-286007.internal/10.140.0.4:8032
20/08/25 17:46:42 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /user/admin/.staging/job_1598350941451_0061
20/08/25 17:46:42 INFO mapred.FileInputFormat: Total input files to process : 1
20/08/25 17:46:43 INFO mapreduce.JobSubmitter: number of splits:2
20/08/25 17:46:43 INFO Configuration.deprecation: yarn.resourcemanager.system-metrics-publisher.enabled is deprecated. Instead, use yarn.system-metrics-publisher.enabled
20/08/25 17:46:43 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1598350941451_0061
20/08/25 17

## Output

In [38]:
!hdfs dfs -ls output/mapreduce/ex1/

Found 3 items
-rw-r--r--   3 admin admin          0 2020-08-25 17:47 output/mapreduce/ex1/_SUCCESS
-rw-r--r--   3 admin admin       1258 2020-08-25 17:47 output/mapreduce/ex1/part-00000
-rw-r--r--   3 admin admin       1286 2020-08-25 17:47 output/mapreduce/ex1/part-00001


In [39]:
!hdfs dfs -cat output/mapreduce/ex1/part-00000

(abe,bear)	5885
(abe,boat)	5968
(abe,can)	3767
(abe,dear)	5317
(abe,fong)	121493
(abe,punpun)	3691
(abe,vate)	6722
(bear,abe)	6023
(bear,boat)	5303
(bear,can)	3414
(bear,dear)	4975
(bear,fong)	6095
(bear,punpun)	3581
(bear,vate)	7107
(boat,abe)	6493
(boat,bear)	5946
(boat,can)	3659
(boat,dear)	4808
(boat,fong)	7016
(boat,punpun)	4058
(boat,vate)	5204
(boss,it)	3278
(boss,kao)	2910
(boss,non)	3505
(boss,por)	3394
(can,abe)	4704
(can,bear)	3772
(can,boat)	3920
(can,dear)	2657
(can,fong)	4808
(can,punpun)	1862
(can,vate)	4104
(dear,abe)	5578
(dear,bear)	4946
(dear,boat)	4804
(dear,can)	3436
(dear,fong)	5541
(dear,punpun)	3300
(dear,vate)	5942
(fong,abe)	117047
(fong,bear)	6289
(fong,boat)	5763
(fong,can)	4544
(fong,dear)	5581
(fong,punpun)	4003
(fong,vate)	6550
(it,boss)	3720
(it,kao)	3029
(it,non)	4627
(it,por)	27077
(kao,boss)	2769
(kao,it)	3903
(kao,non)	2972
(kao,por)	2392
(non,boss)	3404
(non,it)	4089
(non,kao)	3505
(non,por)	3387
(por,boss)	3398
(por,it)	26670
(por,kao)	2778
(por,no

In [40]:
!hdfs dfs -cat output/mapreduce/ex1/part-00001

(abe,boss)	4953
(abe,it)	5509
(abe,kao)	4648
(abe,non)	5249
(abe,por)	5497
(bear,boss)	5101
(bear,it)	31638
(bear,kao)	4210
(bear,non)	5012
(bear,por)	4367
(boat,boss)	4250
(boat,it)	4499
(boat,kao)	3739
(boat,non)	4898
(boat,por)	3678
(boss,abe)	4666
(boss,bear)	3909
(boss,boat)	4405
(boss,can)	2310
(boss,dear)	3710
(boss,fong)	4865
(boss,punpun)	2489
(boss,vate)	5966
(can,boss)	2308
(can,it)	2884
(can,kao)	1922
(can,non)	3311
(can,por)	2932
(dear,boss)	3209
(dear,it)	27233
(dear,kao)	3055
(dear,non)	4222
(dear,por)	3527
(fong,boss)	5406
(fong,it)	5278
(fong,kao)	4382
(fong,non)	5616
(fong,por)	5304
(it,abe)	5674
(it,bear)	33192
(it,boat)	4428
(it,can)	2912
(it,dear)	27830
(it,fong)	4914
(it,punpun)	20058
(it,vate)	5381
(kao,abe)	4639
(kao,bear)	3908
(kao,boat)	3865
(kao,can)	2134
(kao,dear)	3295
(kao,fong)	5233
(kao,punpun)	2244
(kao,vate)	4751
(non,abe)	6074
(non,bear)	4751
(non,boat)	4173
(non,can)	3510
(non,dear)	4199
(non,fong)	5099
(non,punpun)	3162
(non,vate)	5448
(por,abe)	503

# Show that same key belongs to same reducer and each reducer has different keys (ex2) [Same key goes to same reducer]

In [41]:
cat mapreduce/ex2/mapper.py

#!/usr/bin/env python

import sys
for line in sys.stdin:
	# remove leading and trailing whitespace
	line = line.strip()
	rows = line.split(",")
	(name1,phone1,name2,phone2,timestamp_ms,timestamp_str,duration_m) = tuple(rows)
	#print(rows)
	# write the results to STDOUT (standard output);
	# what we output here will be the input for the
	# Reduce step, i.e. the input for reducer.py
	try: 
		print('{}\t{}'.format(name1, int(duration_m)))
	except ValueError as e:
		continue	



In [42]:
cat mapreduce/ex2/reducer.py

#!/usr/bin/env python

from operator import itemgetter
import sys

key = None
value = 0
count = 0

# input comes from STDIN
for line in sys.stdin:
	# remove leading and trailing whitespace
	line = line.strip()
	row = line.split("\t")
	(key,value) = row

	count += 1
	print(count,key,value)
	


In [43]:
!hdfs dfs -rm -r output/mapreduce/ex2

rm: `output/mapreduce/ex2': No such file or directory


In [44]:
!hadoop jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
-files mapreduce/ex2/mapper.py,mapreduce/ex2/reducer.py \
-mapper "python mapper.py" \
-reducer "python reducer.py" \
-input "data/cdr.csv" \
-output "output/mapreduce/ex2" \
-numReduceTasks 2 

packageJobJar: [] [/opt/cloudera/parcels/CDH-6.3.2-1.cdh6.3.2.p0.1605554/jars/hadoop-streaming-3.0.0-cdh6.3.2.jar] /tmp/streamjob7531792753991110456.jar tmpDir=null
20/08/25 17:47:12 INFO client.RMProxy: Connecting to ResourceManager at cdh-all.asia-east1-b.c.kao-project-286007.internal/10.140.0.4:8032
20/08/25 17:47:13 INFO client.RMProxy: Connecting to ResourceManager at cdh-all.asia-east1-b.c.kao-project-286007.internal/10.140.0.4:8032
20/08/25 17:47:13 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /user/admin/.staging/job_1598350941451_0065
20/08/25 17:47:13 INFO mapred.FileInputFormat: Total input files to process : 1
20/08/25 17:47:13 INFO mapreduce.JobSubmitter: number of splits:2
20/08/25 17:47:13 INFO Configuration.deprecation: yarn.resourcemanager.system-metrics-publisher.enabled is deprecated. Instead, use yarn.system-metrics-publisher.enabled
20/08/25 17:47:14 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1598350941451_0065
20/08/25 17

## Output

In [45]:
!hdfs dfs -ls output/mapreduce/ex2/

Found 3 items
-rw-r--r--   3 admin admin          0 2020-08-25 17:47 output/mapreduce/ex2/_SUCCESS
-rw-r--r--   3 admin admin     389839 2020-08-25 17:47 output/mapreduce/ex2/part-00000
-rw-r--r--   3 admin admin     700683 2020-08-25 17:47 output/mapreduce/ex2/part-00001


In [46]:
!hdfs dfs -cat output/mapreduce/ex2/part-00000

(1, 'boss', '9')	
(2, 'boss', '21')	
(3, 'boss', '20')	
(4, 'boss', '28')	
(5, 'boss', '4')	
(6, 'boss', '18')	
(7, 'boss', '4')	
(8, 'boss', '35')	
(9, 'boss', '28')	
(10, 'boss', '27')	
(11, 'boss', '27')	
(12, 'boss', '16')	
(13, 'boss', '21')	
(14, 'boss', '4')	
(15, 'boss', '7')	
(16, 'boss', '26')	
(17, 'boss', '19')	
(18, 'boss', '1')	
(19, 'boss', '13')	
(20, 'boss', '11')	
(21, 'boss', '19')	
(22, 'boss', '18')	
(23, 'boss', '27')	
(24, 'boss', '48')	
(25, 'boss', '44')	
(26, 'boss', '12')	
(27, 'boss', '19')	
(28, 'boss', '9')	
(29, 'boss', '10')	
(30, 'boss', '17')	
(31, 'boss', '28')	
(32, 'boss', '23')	
(33, 'boss', '7')	
(34, 'boss', '6')	
(35, 'boss', '4')	
(36, 'boss', '4')	
(37, 'boss', '15')	
(38, 'boss', '22')	
(39, 'boss', '2')	
(40, 'boss', '23')	
(41, 'boss', '7')	
(42, 'boss', '16')	
(43, 'boss', '24')	
(44, 'boss', '13')	
(45, 'boss', '32')	
(46, 'boss', '21')	
(47, 'boss', '15')	
(48, 'boss', '10')	
(49, 'boss', '20')	
(50, 'boss', '35')	
(51, 'boss', '48')	
(5

(14599, 'non', '17')	
(14600, 'non', '15')	
(14601, 'non', '43')	
(14602, 'non', '1')	
(14603, 'non', '28')	
(14604, 'non', '32')	
(14605, 'non', '1')	
(14606, 'non', '16')	
(14607, 'non', '4')	
(14608, 'non', '18')	
(14609, 'non', '20')	
(14610, 'non', '5')	
(14611, 'non', '34')	
(14612, 'non', '42')	
(14613, 'non', '47')	
(14614, 'non', '0')	
(14615, 'non', '7')	
(14616, 'non', '13')	
(14617, 'non', '42')	
(14618, 'non', '21')	
(14619, 'non', '16')	
(14620, 'non', '25')	
(14621, 'non', '19')	
(14622, 'non', '13')	
(14623, 'non', '27')	
(14624, 'non', '39')	
(14625, 'non', '11')	
(14626, 'non', '43')	
(14627, 'non', '20')	
(14628, 'non', '4')	
(14629, 'non', '8')	
(14630, 'non', '6')	
(14631, 'non', '24')	
(14632, 'non', '11')	
(14633, 'non', '29')	
(14634, 'non', '27')	
(14635, 'non', '3')	
(14636, 'non', '39')	
(14637, 'non', '29')	
(14638, 'non', '19')	
(14639, 'non', '35')	
(14640, 'non', '6')	
(14641, 'non', '4')	
(14642, 'non', '14')	
(14643, 'non', '40')	
(14644, 'non', '35')	


In [47]:
!hdfs dfs -cat output/mapreduce/ex2/part-00001

(1, 'abe', '16')	
(2, 'abe', '5')	
(3, 'abe', '77')	
(4, 'abe', '9')	
(5, 'abe', '53')	
(6, 'abe', '17')	
(7, 'abe', '41')	
(8, 'abe', '24')	
(9, 'abe', '0')	
(10, 'abe', '35')	
(11, 'abe', '15')	
(12, 'abe', '21')	
(13, 'abe', '21')	
(14, 'abe', '28')	
(15, 'abe', '12')	
(16, 'abe', '27')	
(17, 'abe', '34')	
(18, 'abe', '9')	
(19, 'abe', '15')	
(20, 'abe', '47')	
(21, 'abe', '30')	
(22, 'abe', '44')	
(23, 'abe', '37')	
(24, 'abe', '23')	
(25, 'abe', '1')	
(26, 'abe', '7')	
(27, 'abe', '17')	
(28, 'abe', '45')	
(29, 'abe', '54')	
(30, 'abe', '5')	
(31, 'abe', '20')	
(32, 'abe', '28')	
(33, 'abe', '44')	
(34, 'abe', '20')	
(35, 'abe', '13')	
(36, 'abe', '5')	
(37, 'abe', '4')	
(38, 'abe', '2')	
(39, 'abe', '44')	
(40, 'abe', '56')	
(41, 'abe', '49')	
(42, 'abe', '44')	
(43, 'abe', '19')	
(44, 'abe', '1')	
(45, 'abe', '40')	
(46, 'abe', '19')	
(47, 'abe', '53')	
(48, 'abe', '37')	
(49, 'abe', '3')	
(50, 'abe', '48')	
(51, 'abe', '7')	
(52, 'abe', '17')	
(53, 'abe', '46')	
(54, 'abe', '38

(13974, 'can', '4')	
(13975, 'can', '35')	
(13976, 'can', '17')	
(13977, 'can', '15')	
(13978, 'can', '8')	
(13979, 'can', '22')	
(13980, 'can', '12')	
(13981, 'can', '33')	
(13982, 'can', '19')	
(13983, 'can', '21')	
(13984, 'can', '21')	
(13985, 'can', '27')	
(13986, 'can', '9')	
(13987, 'can', '3')	
(13988, 'can', '2')	
(13989, 'can', '8')	
(13990, 'can', '22')	
(13991, 'can', '1')	
(13992, 'can', '50')	
(13993, 'can', '14')	
(13994, 'can', '21')	
(13995, 'can', '38')	
(13996, 'can', '22')	
(13997, 'can', '24')	
(13998, 'can', '9')	
(13999, 'can', '11')	
(14000, 'can', '15')	
(14001, 'can', '33')	
(14002, 'can', '23')	
(14003, 'can', '3')	
(14004, 'can', '16')	
(14005, 'can', '13')	
(14006, 'can', '2')	
(14007, 'can', '5')	
(14008, 'can', '35')	
(14009, 'can', '16')	
(14010, 'can', '5')	
(14011, 'can', '9')	
(14012, 'can', '4')	
(14013, 'can', '9')	
(14014, 'can', '5')	
(14015, 'can', '24')	
(14016, 'can', '23')	
(14017, 'can', '1')	
(14018, 'can', '27')	
(14019, 'can', '9')	
(14020

(27759, 'punpun', '3')	
(27760, 'punpun', '13')	
(27761, 'punpun', '4')	
(27762, 'punpun', '0')	
(27763, 'punpun', '2')	
(27764, 'punpun', '17')	
(27765, 'punpun', '8')	
(27766, 'punpun', '1')	
(27767, 'punpun', '0')	
(27768, 'punpun', '18')	
(27769, 'punpun', '42')	
(27770, 'punpun', '23')	
(27771, 'punpun', '42')	
(27772, 'punpun', '24')	
(27773, 'punpun', '8')	
(27774, 'punpun', '7')	
(27775, 'punpun', '8')	
(27776, 'punpun', '2')	
(27777, 'punpun', '9')	
(27778, 'punpun', '20')	
(27779, 'punpun', '6')	
(27780, 'punpun', '18')	
(27781, 'punpun', '11')	
(27782, 'punpun', '44')	
(27783, 'punpun', '12')	
(27784, 'punpun', '12')	
(27785, 'punpun', '2')	
(27786, 'punpun', '17')	
(27787, 'punpun', '16')	
(27788, 'punpun', '10')	
(27789, 'punpun', '8')	
(27790, 'punpun', '8')	
(27791, 'punpun', '7')	
(27792, 'punpun', '14')	
(27793, 'punpun', '26')	
(27794, 'punpun', '4')	
(27795, 'punpun', '16')	
(27796, 'punpun', '24')	
(27797, 'punpun', '43')	
(27798, 'punpun', '12')	
(27799, 'punpun', 

# Find average couple call duration each year (ex3)

In [48]:
cat mapreduce/ex3/mapper.py

#!/usr/bin/env python

import sys
for line in sys.stdin:
	# remove leading and trailing whitespace
	line = line.strip()
	rows = line.split(",")
	(name1,phone1,name2,phone2,timestamp_ms,timestamp_str,duration_m) = tuple(rows)
	#print(rows)
	# write the results to STDOUT (standard output);
	# what we output here will be the input for the
	# Reduce step, i.e. the input for reducer.py
	try: 
		print('({},{})\t{}'.format(name1,name2, int(duration_m)))
	except ValueError as e:
		continue	



In [49]:
cat mapreduce/ex3/reducer.py

#!/usr/bin/env python

from operator import itemgetter
import sys

current_key = None
sum = 0
count = 0

# input comes from STDIN
for line in sys.stdin:
	# remove leading and trailing whitespace
	line = line.strip()
	row = line.split("\t")
	(key,value) = row

	try: 
		value = int(value)
	except ValueError:
		continue

	# this IF-switch only works because Hadoop sorts map output
	# by key (here: word) before it is passed to the reducer
	if current_key == (key):
		sum += value
		count += 1
	else:
		if current_key:
	 		# write result to STDOUT
			print('{}\t{}'.format(current_key,float(sum) / count))
		current_key = key
		sum = value
		count = 1


if current_key == key:
	print('{}\t{}'.format(current_key,float(sum) / count))



In [50]:
!hdfs dfs -rm -r output/mapreduce/ex3

rm: `output/mapreduce/ex3': No such file or directory


In [51]:
!hadoop jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
-files mapreduce/ex3/mapper.py,mapreduce/ex3/reducer.py \
-mapper "python mapper.py" \
-reducer "python reducer.py" \
-input "data/cdr.csv" \
-output "output/mapreduce/ex3" \
-numReduceTasks 2 

packageJobJar: [] [/opt/cloudera/parcels/CDH-6.3.2-1.cdh6.3.2.p0.1605554/jars/hadoop-streaming-3.0.0-cdh6.3.2.jar] /tmp/streamjob6721974419162819223.jar tmpDir=null
20/08/25 17:47:45 INFO client.RMProxy: Connecting to ResourceManager at cdh-all.asia-east1-b.c.kao-project-286007.internal/10.140.0.4:8032
20/08/25 17:47:45 INFO client.RMProxy: Connecting to ResourceManager at cdh-all.asia-east1-b.c.kao-project-286007.internal/10.140.0.4:8032
20/08/25 17:47:45 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /user/admin/.staging/job_1598350941451_0068
20/08/25 17:47:46 INFO mapred.FileInputFormat: Total input files to process : 1
20/08/25 17:47:46 INFO mapreduce.JobSubmitter: number of splits:2
20/08/25 17:47:46 INFO Configuration.deprecation: yarn.resourcemanager.system-metrics-publisher.enabled is deprecated. Instead, use yarn.system-metrics-publisher.enabled
20/08/25 17:47:46 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1598350941451_0068
20/08/25 17

## Output

In [52]:
!hdfs dfs -ls output/mapreduce/ex3/

Found 3 items
-rw-r--r--   3 admin admin          0 2020-08-25 17:48 output/mapreduce/ex3/_SUCCESS
-rw-r--r--   3 admin admin       1930 2020-08-25 17:48 output/mapreduce/ex3/part-00000
-rw-r--r--   3 admin admin       1961 2020-08-25 17:48 output/mapreduce/ex3/part-00001


In [53]:
!hdfs dfs -cat output/mapreduce/ex3/part-00000

(abe,bear)	26.995412844
(abe,boat)	26.7623318386
(abe,can)	18.7412935323
(abe,dear)	25.1990521327
(abe,fong)	32.1070295983
(abe,punpun)	18.8316326531
(abe,vate)	32.0095238095
(bear,abe)	30.4191919192
(bear,boat)	24.2146118721
(bear,can)	16.7352941176
(bear,dear)	23.3568075117
(bear,fong)	28.3488372093
(bear,punpun)	16.1306306306
(bear,vate)	29.8613445378
(boat,abe)	27.5127118644
(boat,bear)	25.1949152542
(boat,can)	16.8617511521
(boat,dear)	23.4536585366
(boat,fong)	29.112033195
(boat,punpun)	16.9083333333
(boat,vate)	24.663507109
(boss,it)	18.312849162
(boss,kao)	12.3305084746
(boss,non)	16.9323671498
(boss,por)	16.2392344498
(can,abe)	21.2850678733
(can,bear)	19.1472081218
(can,boat)	17.7375565611
(can,dear)	15.011299435
(can,fong)	21.6576576577
(can,punpun)	8.46363636364
(can,vate)	21.2642487047
(dear,abe)	26.8173076923
(dear,bear)	21.5982532751
(dear,boat)	20.3559322034
(dear,can)	15.3392857143
(dear,fong)	26.8980582524
(dear,punpun)	13.0952380952
(dear,vate)	24.8619246862
(fong,ab

In [54]:
!hdfs dfs -cat output/mapreduce/ex3/part-00001

(abe,boss)	22.2107623318
(abe,it)	25.9858490566
(abe,kao)	21.4193548387
(abe,non)	25.1148325359
(abe,por)	23.2923728814
(bear,boss)	21.5232067511
(bear,it)	23.1441111924
(bear,kao)	19.765258216
(bear,non)	23.641509434
(bear,por)	21.5123152709
(boat,boss)	18.8888888889
(boat,it)	22.8375634518
(boat,kao)	17.4719626168
(boat,non)	22.3652968037
(boat,por)	20.0983606557
(boss,abe)	22.6504854369
(boss,bear)	18.7932692308
(boss,boat)	20.2064220183
(boss,can)	11.4356435644
(boss,dear)	17.5829383886
(boss,fong)	23.7317073171
(boss,punpun)	11.0622222222
(boss,vate)	26.1666666667
(can,boss)	11.3694581281
(can,it)	14.5656565657
(can,kao)	9.75634517766
(can,non)	14.2103004292
(can,por)	13.6372093023
(dear,boss)	16.3724489796
(dear,it)	19.4382583869
(dear,kao)	15.275
(dear,non)	19.1909090909
(dear,por)	17.2048780488
(fong,boss)	24.2421524664
(fong,it)	25.4975845411
(fong,kao)	22.1313131313
(fong,non)	26.2429906542
(fong,por)	24.6697674419
(it,abe)	26.0275229358
(it,bear)	24.0
(it,boat)	20.7887323944